In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn import svm
from Utils import GeoUtils

from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.multioutput import MultiOutputRegressor

%matplotlib inline

In [8]:
data = pd.read_csv("Dados/LocTreino_Equipe_4.csv")

bts_data = pd.read_csv("Dados/dados_BTSs.csv")

In [9]:
data.drop_duplicates(inplace=True)
data.shape
data = data.drop(['pontoId'], axis=1)
data['taBTS2'] = data['taBTS2'].fillna(data['taBTS2'].mean())
data['taBTS4'] = data['taBTS4'].fillna(data['taBTS4'].mean())
data.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,-8.070097,-34.894814,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,0.004232,0.004354,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,-8.077570,-34.906830,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,-8.073197,-34.897387,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,-8.069940,-34.895346,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,-8.067113,-34.891169,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,-8.059339,-34.885067,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [10]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [11]:
x = data.drop(['lat', 'lon'], axis=1)
x.describe()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0000
mean,116.654370,119.101165,123.863545,115.814052,121.320526,125.656192,0.663333,1.008733,2.525333,1.099388,1.093333,1.0560
std,15.229664,12.666188,9.031316,12.414620,12.552702,13.134365,0.698317,0.863886,0.967117,1.954747,0.856998,0.6692
min,86.256667,77.256667,88.340000,84.056667,79.523333,89.856667,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000
25%,106.827500,111.673333,119.190000,106.583750,112.823333,116.077500,0.000000,0.000000,2.000000,0.000000,0.000000,1.0000
50%,117.156667,120.690000,124.273333,116.273333,122.840000,128.756667,1.000000,1.000000,3.000000,1.000000,1.000000,1.0000
75%,124.790000,128.808750,130.117083,125.590000,130.423333,135.996250,1.000000,1.000000,3.000000,1.000000,2.000000,2.0000
max,476.882449,144.890000,145.890000,145.423333,146.890000,150.123333,3.000000,11.090452,5.000000,38.747484,3.000000,2.0000


In [12]:
y = data[['lat', 'lon']]
y.describe()

,lat,lon
count,1500.000000,1500.000000
mean,-8.070097,-34.894814
std,0.004232,0.004354
min,-8.077570,-34.906830
25%,-8.073197,-34.897387
50%,-8.069940,-34.895346
75%,-8.067113,-34.891169
max,-8.059339,-34.885067


In [35]:
transformerY = RobustScaler().fit(y)
y_scaled = transformerY.transform(y)

transformerX = RobustScaler().fit(x)
x_scaled = transformerX.transform(x)

X_train, X_test, y_train, y_test = train_test_split(x, y_scaled, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1050, 12) (450, 12) (1050, 2) (450, 2)


In [36]:
multioutputregressor = MultiOutputRegressor(model_xgb).fit(X_train, y_train)

In [37]:
print (transformerY.inverse_transform(multioutputregressor.predict(X_test)))

[[ -8.077583  -34.89902  ]
 [ -8.068533  -34.89007  ]
 [ -8.065399  -34.88707  ]
 [ -8.075666  -34.899967 ]
 [ -8.069289  -34.88988  ]
 [ -8.068516  -34.897007 ]
 [ -8.076502  -34.896885 ]
 [ -8.064201  -34.885956 ]
 [ -8.070244  -34.896904 ]
 [ -8.072661  -34.895794 ]
 [ -8.066912  -34.893772 ]
 [ -8.073889  -34.89636  ]
 [ -8.077369  -34.898834 ]
 [ -8.0646105 -34.900723 ]
 [ -8.069543  -34.88996  ]
 [ -8.062009  -34.90228  ]
 [ -8.069848  -34.893764 ]
 [ -8.076465  -34.89988  ]
 [ -8.069259  -34.890682 ]
 [ -8.072971  -34.896523 ]
 [ -8.069337  -34.89421  ]
 [ -8.072275  -34.89198  ]
 [ -8.065219  -34.893036 ]
 [ -8.069198  -34.888767 ]
 [ -8.07088   -34.89167  ]
 [ -8.070691  -34.892593 ]
 [ -8.072482  -34.896835 ]
 [ -8.065504  -34.900867 ]
 [ -8.072893  -34.8961   ]
 [ -8.069385  -34.89026  ]
 [ -8.072308  -34.900665 ]
 [ -8.071388  -34.900524 ]
 [ -8.062274  -34.88806  ]
 [ -8.069826  -34.890923 ]
 [ -8.071308  -34.891792 ]
 [ -8.065171  -34.900677 ]
 [ -8.069411  -34.889942 ]
 

In [38]:
print (np.mean((transformerY.inverse_transform(multioutputregressor.predict(X_test)) - transformerY.inverse_transform(y_test))**2, axis=0))

[3.47289100e-07 3.85550428e-07]


In [40]:
v = transformerY.inverse_transform(multioutputregressor.predict(X_test))

lat1 = v[:,0]
lon1 = v[:,1]
lat2 = transformerY.inverse_transform(y_test)[:,0]
lon2 = transformerY.inverse_transform(y_test)[:,1]

dist2 = GeoUtils.GeoUtils.distanceInKm(lat1, lon1,lat2, lon2)

print(np.array(dist).mean())

0.08926488438237418


In [41]:
svr = svm.SVR()
regr = MultiOutputRegressor(svr)
regr.fit(X_train,y_train)

out2 = transformerY.inverse_transform(regr.predict(X_test))

lat3 = out2[:,0]
lon3 = out2[:,1]
lat4 = transformerY.inverse_transform(y_test)[:,0]
lon4 = transformerY.inverse_transform(y_test)[:,1]

dist2 = GeoUtils.GeoUtils.distanceInKm(lat3, lon3,lat4, lon4)
print(np.array(dist2).mean())

0.5011747062861148


In [20]:
data2 = pd.read_csv("Dados/LocTreino_Equipe_8.csv")

data2.drop_duplicates(inplace=True)
data2.shape
data2 = data2.drop(['pontoId'], axis=1)
data2['taBTS2'] = data2['taBTS2'].fillna(data2['taBTS2'].mean())
data2['taBTS4'] = data2['taBTS4'].fillna(data2['taBTS4'].mean())
data2['taBTS6'] = data2['taBTS6'].fillna(data2['taBTS6'].mean())
data2['pathBTS5'] = data2['pathBTS5'].fillna(data2['pathBTS5'].mean())

x2 = data2.drop(['lat', 'lon'], axis=1)
y2 = data2[['lat', 'lon']]

data2.describe()

,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,-8.070134,-34.895007,116.751773,118.649758,123.580585,115.984134,122.759922,125.123635,0.662000,0.996000,2.504168,0.996000,1.115333,1.066769
std,0.004245,0.004256,12.197069,12.745641,9.149696,12.453865,25.765837,13.512854,0.695758,0.827309,1.077002,0.827309,0.830958,1.094289
min,-8.077570,-34.906830,86.256667,76.423333,89.123333,83.640000,80.123333,89.823333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,-8.073231,-34.897486,107.181667,110.840000,118.856667,106.848333,113.554583,114.881667,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000
50%,-8.069965,-34.895634,117.510833,119.865000,124.173333,116.290000,123.406667,128.306667,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
75%,-8.067056,-34.891804,125.244167,128.365000,129.723333,125.646250,130.431667,135.871250,1.000000,1.000000,3.000000,1.000000,2.000000,2.000000
max,-8.059338,-34.885067,144.290000,143.423333,145.890000,145.456667,664.317682,149.756667,3.000000,4.000000,18.918054,4.000000,3.000000,34.086073


In [21]:
transformerY2 = RobustScaler().fit(y2)
y2_scaled = transformerY2.transform(y2)

In [22]:
print (np.mean((transformerY2.inverse_transform(multioutputregressor.predict(x2)) - transformerY2.inverse_transform(y2_scaled))**2, axis=0))

[2.76381396e-07 6.34930205e-07]


In [23]:
transformerY2.inverse_transform(multioutputregressor.predict(x2))

array([[ -8.069407, -34.895855],
       [ -8.066186, -34.88989 ],
       [ -8.06998 , -34.89905 ],
       ...,
       [ -8.072911, -34.900364],
       [ -8.068869, -34.88989 ],
       [ -8.066358, -34.894306]], dtype=float32)

In [28]:
p = transformerY2.inverse_transform(multioutputregressor.predict(x2))

latv = p[:,0]
lonv = p[:,1]
latv2 = transformerY2.inverse_transform(y2_scaled)[:,0]
lonv2 = transformerY2.inverse_transform(y2_scaled)[:,1]

dist = GeoUtils.GeoUtils.distanceInKm(latv, lonv,latv2, lonv2)

print(np.array(dist).mean())

0.08926488438237418
